## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Maldonado,UY,-34.6667,-54.9167,51.19,92,0,5.41,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,42.91,81,75,17.27,broken clouds
2,2,Litovko,RU,49.2361,135.1742,43.95,39,21,16.46,few clouds
3,3,Quepos,CR,9.4236,-84.1652,76.46,89,100,0.92,overcast clouds
4,4,Jamestown,US,42.0970,-79.2353,49.64,67,0,10.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Quepos,CR,9.4236,-84.1652,76.46,89,100,0.92,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,70.32,61,91,23.91,overcast clouds
6,6,Mayumba,GA,-3.4320,10.6554,75.13,86,100,3.18,light rain
7,7,Merauke,ID,-8.4667,140.3333,83.34,79,100,14.67,overcast clouds
8,8,Richards Bay,ZA,-28.7830,32.0377,69.71,69,75,9.75,broken clouds
10,10,Tessalit,ML,20.1986,1.0114,85.75,20,7,9.51,clear sky
11,11,Karratha,AU,-20.7377,116.8463,80.35,36,0,14.99,clear sky
12,12,Malakal,SS,9.5334,31.6605,72.91,92,87,0.13,overcast clouds
15,15,Talara,PE,-4.5772,-81.2719,63.30,81,75,16.71,broken clouds
18,18,Butaritari,KI,3.0707,172.7902,82.26,72,66,17.92,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df= preferred_cities_df.dropna(axis="index", how="any")

clean_preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Quepos,CR,9.4236,-84.1652,76.46,89,100,0.92,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,70.32,61,91,23.91,overcast clouds
6,6,Mayumba,GA,-3.4320,10.6554,75.13,86,100,3.18,light rain
7,7,Merauke,ID,-8.4667,140.3333,83.34,79,100,14.67,overcast clouds
8,8,Richards Bay,ZA,-28.7830,32.0377,69.71,69,75,9.75,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Quepos,CR,76.46,overcast clouds,9.4236,-84.1652,
5,Rikitea,PF,70.32,overcast clouds,-23.1203,-134.9692,
6,Mayumba,GA,75.13,light rain,-3.4320,10.6554,
7,Merauke,ID,83.34,overcast clouds,-8.4667,140.3333,
8,Richards Bay,ZA,69.71,broken clouds,-28.7830,32.0377,
10,Tessalit,ML,85.75,clear sky,20.1986,1.0114,
11,Karratha,AU,80.35,clear sky,-20.7377,116.8463,
12,Malakal,SS,72.91,overcast clouds,9.5334,31.6605,
15,Talara,PE,63.30,broken clouds,-4.5772,-81.2719,
18,Butaritari,KI,82.26,broken clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...Skipping.")
    
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Quepos,CR,76.46,overcast clouds,9.4236,-84.1652,Jungle Beach Hotel at Manuel Antonio
5,Rikitea,PF,70.32,overcast clouds,-23.1203,-134.9692,People ThankYou
6,Mayumba,GA,75.13,light rain,-3.4320,10.6554,LIKWALE LODGE
7,Merauke,ID,83.34,overcast clouds,-8.4667,140.3333,NaN
8,Richards Bay,ZA,69.71,broken clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('',np.nan, inplace=True)
hotel_df.dropna().head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Quepos,CR,76.46,overcast clouds,9.4236,-84.1652,Jungle Beach Hotel at Manuel Antonio
5,Rikitea,PF,70.32,overcast clouds,-23.1203,-134.9692,People ThankYou
6,Mayumba,GA,75.13,light rain,-3.4320,10.6554,LIKWALE LODGE
8,Richards Bay,ZA,69.71,broken clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
11,Karratha,AU,80.35,clear sky,-20.7377,116.8463,ibis Styles Karratha


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description} </dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
# max_temp = weather_vacation_df["Max Temp"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer=gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
try:
    fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             #dissipating=False, max_intensity=300, point_radius=4)
#marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
#fig.add_layer(marker_layer)
# 11b. Display the figure
    fig
except Exception as e:
    print(e)